# Stacking
Stacking is an ensemble method that creates a strong metamodel trained on the predictions of several independent base models.

- Comparison with Boosting and Bagging:
    * Training Data:
        - Stacking uses the full training set for both base models and the metamodel.
        - Boosting and bagging use sampling techniques to create training sets for their base models.
    * Base Models:
        - In stacking, base models can be different algorithms (e.g., logistic regression, random forest), leveraging their unique strengths.
        - Boosting and bagging often use a large number of similar base models (sometimes hundreds).
    * Metamodel Complexity:
        - Stacking builds a more complex metamodel that learns from the base model predictions.
        - Boosting and bagging typically use naive metamodels that average or vote on predictions.
- Functionality of the Metamodel:
The metamodel can prioritize and weight the contributions of different base models based on their unique insights.
Additional data can be incorporated into the metamodel alongside base model predictions, depending on the problem.
- Advantages of Stacking:
Utilizing different algorithms allows for diverse insights from the training data.
The framework combines outputs from various models, enhancing predictive power.

- Stacking starts with the full training set, generating predictions from different base models that are then fed into a metamodel.
The metamodel is trained to produce the final prediction, learning how to weight the insights from the base models effectively.
This structure allows stacking to be a powerful framework in ensemble learning, capturing a wider range of information for improved predictions.


In [6]:
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier, RandomForestClassifier



In [ ]:
# Check the hyperparameters

estimators = [{'gb', GradietBoost
StackingClassifier().get_params()